<a href="https://colab.research.google.com/github/IA1819/entregable2-lucabem/blob/master/AlgoritmoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [64]:
from google.colab import files
uploaded = files.upload()

Saving sample.csv to sample (1).csv
Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [0]:
import io
df_sample = pd.read_csv(io.BytesIO(uploaded['sample.csv']))
df_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [0]:
df_train['Prediction'] = df_train['Prediction'].astype('int64')

In [0]:
d = {'Male' : 1, 'Female' : 0}
df_train['Gender'] = df_train['Gender'].map(d)
df_test['Gender'] = df_test['Gender'].map(d)

Como hemos visto en la matriz de correlaccion, habia varios descriptores que mantenian una relaccion lineal. En este caso, eliminamos esos descriptores ya que ofrecen informacion muy parecida.

In [0]:
df_train.drop(['Albumin_and_Globulin_Ratio'], axis=1, inplace=True)
df_test.drop(['Albumin_and_Globulin_Ratio'], axis=1, inplace=True)

In [0]:
df_train.drop(['Albumin'], axis=1, inplace=True)
df_test.drop(['Albumin'], axis=1, inplace=True)

In [0]:
df_train.drop(['Direct_Bilirubin'], axis=1, inplace=True)
df_test.drop(['Direct_Bilirubin'], axis=1, inplace=True)

In [0]:
X =  df_train.values[:,:-1]
Y =  df_train.values[:,-1]
Z =  df_test.values

In [72]:
Z

array([[ 36. ,   1. ,   0.8, ...,  29. ,  39. ,   6. ],
       [ 55. ,   1. ,  14.1, ...,  35. ,  63. ,   5. ],
       [ 61. ,   1. ,   0.8, ...,  28. ,  35. ,   6.9],
       ...,
       [ 60. ,   1. ,   5.2, ..., 126. , 202. ,   6.8],
       [ 72. ,   1. ,   0.8, ...,  23. ,  35. ,   6. ],
       [ 27. ,   1. ,   1. , ..., 137. , 145. ,   6. ]])

In [0]:
from sklearn.model_selection import train_test_split

**Estandarizamos los valores**

Usando la estandarización se consiguen centrar las columnas de descriptores con media 0 y desviación típica 1 de modo que las columnas de descriptores toman la forma de una distribución normal lo que hace que se aprendan de manera más sencilla los pesos.

In [0]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X)
Z_test = stdsc.fit_transform(Z)

In [75]:
Z_test

array([[-0.53501204,  0.52981294, -0.44385737, ..., -0.37725189,
        -0.38476147, -0.3194825 ],
       [ 0.68348008,  0.52981294,  1.89885056, ..., -0.32297818,
        -0.22566256, -1.20964367],
       [ 1.06826706,  0.52981294, -0.44385737, ..., -0.38629751,
        -0.41127796,  0.48166255],
       ...,
       [ 1.0041359 ,  0.52981294,  0.33117382, ...,  0.50017319,
         0.69578532,  0.39264643],
       [ 1.77370986,  0.52981294, -0.44385737, ..., -0.43152561,
        -0.41127796, -0.3194825 ],
       [-1.11219252,  0.52981294, -0.40862868, ...,  0.599675  ,
         0.3179254 , -0.3194825 ]])

**Entrenamos el modelo**

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
mejorLog = LogisticRegression(C=1000, penalty='l2')

In [0]:
(trainData, testData, trainLabels, testLabels) = train_test_split(X_train_std, Y, test_size=0.25)

In [113]:
mejorLog.fit(trainData, trainLabels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
def evalua(clasificador,testData,testLabels):
  contador = 0
  for i in range(0,len(testData)):
    if clasificador.predict([testData[i]]) == testLabels[i]:
      contador = contador+1
  return contador

In [115]:
evalua(mejorLog, testData, testLabels)/len(testData)

0.7363636363636363

**Realizamos la prediccion**

In [0]:
predicciones = mejorLog.predict(Z_test)

In [0]:
predicciones = [True if prediccion==1 else False for prediccion in predicciones]

In [0]:
soldf = pd.DataFrame(list(enumerate(predicciones)))

In [0]:
soldf.columns = ['Id','Prediction']

In [120]:
soldf.describe(include=['object', 'bool'])

,Prediction
count,146
unique,2
top,False
freq,100


**Obtenemos el .csv**

In [0]:
soldf.to_csv("submissionLR_SinAlbumina_SinDB.csv",sep=',',index=False)

In [0]:
from google.colab import files
files.download("submissionLR_SinAlbumina_SinDB.csv")